In [1]:
!/opt/conda/envs/rapids/bin/python -m pip install transformers[torch]

  Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl (447 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl (906.4 MB)
  Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (13.8 MB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-no

In [2]:
!/opt/conda/envs/rapids/bin/python -m pip install torch
!/opt/conda/envs/rapids/bin/python -m pip install transformers

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

In [4]:
# Tokenize the 'text' column and keep the labels (readmitted_within_30_days, OUD, short_term_mortality)
data = pd.read_csv(r'Data/NOTEEVENTS.csv',compression = 'gzip')
addmissions = pd.read_csv(r'Data/ADMISSIONS.csv',compression = 'gzip')

/tmp/ipykernel_4006585/632160543.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'Data/NOTEEVENTS.csv',compression = 'gzip')


In [5]:
data.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [6]:
data.columns = data.columns.str.lower()
addmissions.columns = addmissions.columns.str.lower()

In [7]:
data = pd.merge(data,addmissions[['subject_id','hadm_id','dischtime','hospital_expire_flag','ethnicity','religion']],on = ['subject_id','hadm_id'],how = 'left')

In [8]:
data[['chartdate','dischtime']] = data[['chartdate','dischtime']].apply(pd.to_datetime,errors = 'coerce')
data['dischtime'] = data['dischtime'] - pd.to_timedelta(1, unit='d')
data = data[data['dischtime'] >= data['chartdate']]
data.reset_index(inplace = True,drop = True)
data = data[['subject_id', 'text', 'ethnicity', 'religion', 'hospital_expire_flag']]

In [9]:
data = pd.get_dummies(data, columns=['ethnicity', 'religion'])


In [10]:
if torch.cuda.is_available():
    print(f"PyTorch is using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("PyTorch is using CPU")

PyTorch is using GPU: NVIDIA A100-SXM4-80GB


In [11]:

data['text'] = data['text'].fillna('')  # Handle missing text

# Define labels

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

# data = data.sample(1000)

In [12]:
labels_data = data['hospital_expire_flag']
patient_ids = data['subject_id'].values
print(len(labels_data))

1738125


In [13]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import time
# from transformers import AutoTokenizer

# # Initialize the tokenizer
# # tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

# # Parameters
# batch_size = 100  # Adjust based on your data and system memory
# total_batches = len(data['text']) // batch_size + (1 if len(data['text']) % batch_size != 0 else 0)
# batches = [data['text'][i:i + batch_size].tolist() for i in range(0, len(data['text']), batch_size)]


# tokenized_data_batches = []
# tokenized_count = 0
# start_time = time.time()
# last_print_time = start_time

# # Define tokenization function
# def tokenize_batch(batch):
#     return tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

# with ThreadPoolExecutor(max_workers=5) as executor:
#     futures = {executor.submit(tokenize_batch, batch): i for i, batch in enumerate(batches)}

#     for future in as_completed(futures):
#         # Retrieve the tokenized batch and add to results
#         tokenized_data_batches.append(future.result())
#         tokenized_count += 1
        
#         # Calculate and print progress every 5 seconds
#         current_time = time.time()
#         if current_time - last_print_time >= 5:
#             progress = (tokenized_count / total_batches) * 100
#             print(f"Progress: {progress:.2f}% of data tokenized")
#             last_print_time = current_time

# print("Tokenization complete.")
# print(f"Total time: {time.time() - start_time:.2f} seconds")


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
import psutil  # For memory monitoring

# Define device


# Set a fixed maximum sequence length
max_length = 512

# Tokenize the data with fixed max_length padding and truncation
tokenized_data_batches = []
batch_size = 2000  # Adjust based on system's memory
total_batches = len(data['text']) // batch_size + (1 if len(data['text']) % batch_size > 0 else 0)  # Total number of batches

# Function to monitor memory usage
def log_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Memory Usage: {memory_info.rss / (1024 ** 2):.2f} MB (RSS), {memory_info.vms / (1024 ** 2):.2f} MB (VMS)")

for i in range(0, len(data['text']), batch_size):
    batch_number = i // batch_size + 1
    print(f"Processing batch {batch_number}/{total_batches} ({(batch_number / total_batches) * 100:.2f}% completed)")
    log_memory_usage()  # Log memory usage before processing the batch
    
    batch = data['text'][i:i + batch_size].tolist()
    tokenized_batch = tokenizer(batch, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    tokenized_data_batches.append(tokenized_batch)
    
    log_memory_usage()  # Log memory usage after processing the batch

# Concatenate tokenized batches into single tensors
all_input_ids = torch.cat([batch['input_ids'] for batch in tokenized_data_batches], dim=0)
all_attention_masks = torch.cat([batch['attention_mask'] for batch in tokenized_data_batches], dim=0)

# Combine into a single dictionary if needed
merged_tokenized_data = {
    'input_ids': all_input_ids,
    'attention_mask': all_attention_masks
}

# Confirm tensor shapes
print("Merged input_ids shape:", merged_tokenized_data['input_ids'].shape)
print("Merged attention_mask shape:", merged_tokenized_data['attention_mask'].shape)

# Define labels tensor for classification
labels_tensor = torch.tensor(labels_data.values, dtype=torch.long)

# Final memory usage log
log_memory_usage()


Processing batch 1/870 (0.11% completed)
Memory Usage: 5783.86 MB (RSS), 18280.98 MB (VMS)
Memory Usage: 5847.27 MB (RSS), 18342.52 MB (VMS)
Processing batch 2/870 (0.23% completed)
Memory Usage: 5847.27 MB (RSS), 18342.52 MB (VMS)
Memory Usage: 5871.60 MB (RSS), 18366.95 MB (VMS)
Processing batch 3/870 (0.34% completed)
Memory Usage: 5871.60 MB (RSS), 18366.95 MB (VMS)
Memory Usage: 5887.23 MB (RSS), 18382.58 MB (VMS)
Processing batch 4/870 (0.46% completed)
Memory Usage: 5887.23 MB (RSS), 18382.58 MB (VMS)
Memory Usage: 5910.66 MB (RSS), 18406.02 MB (VMS)
Processing batch 5/870 (0.57% completed)
Memory Usage: 5910.66 MB (RSS), 18406.02 MB (VMS)
Memory Usage: 5934.10 MB (RSS), 18429.45 MB (VMS)
Processing batch 6/870 (0.69% completed)
Memory Usage: 5934.10 MB (RSS), 18429.45 MB (VMS)
Memory Usage: 5957.32 MB (RSS), 18452.89 MB (VMS)
Processing batch 7/870 (0.80% completed)
Memory Usage: 5957.32 MB (RSS), 18452.89 MB (VMS)
Memory Usage: 5980.98 MB (RSS), 18476.33 MB (VMS)
Processing ba

In [23]:
all_attention_masks.shape

torch.Size([1738125, 512])

In [22]:
torch.save(all_input_ids, 'input_ids.pt')
torch.save(all_attention_masks, 'attention_mask.pt')
torch.save(labels_tensor, 'labels_tensor.pt')

In [13]:
all_input_ids = torch.load('input_ids.pt')
all_attention_masks = torch.load('attention_mask.pt')
labels_tensor = torch.load('labels_tensor.pt')

/tmp/ipykernel_4006585/1263569302.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_input_ids = torch.load('input_ids.pt')
/tmp/ipykernel_4006585/1263569302.py:2: Futu

In [16]:

# Split data into train and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels, train_patient_ids, val_patient_ids = train_test_split(
    all_input_ids,
    all_attention_masks,
    labels_tensor,
    patient_ids,
    test_size=0.1,
    random_state=42
)

# Define custom dataset class for the Trainer
class MortalityDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels, patient_ids):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels
        self.patient_ids = patient_ids

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx],
            'patient_id': self.patient_ids[idx]
        }

# Create train and validation datasets
train_dataset = MortalityDataset(train_inputs, train_masks, train_labels, train_patient_ids)
val_dataset = MortalityDataset(val_inputs, val_masks, val_labels, val_patient_ids)

# Define and configure the model
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)
model.to(device)  # Move to GPU if available

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',          # Output directory
#     evaluation_strategy="epoch",     # Evaluate at each epoch
#     per_device_train_batch_size=8,   # Batch size for training
#     per_device_eval_batch_size=8,    # Batch size for evaluation
#     num_train_epochs=3,              # Number of epochs
#     weight_decay=0.01,               # Strength of weight decay
#     logging_dir='./logs',            # Directory for storing logs
#     logging_steps=10,
# )

# # Initialize Trainer
# trainer = Trainer(
#     model=model,                        
#     args=training_args,            
#     train_dataset=train_dataset,       
#     eval_dataset=val_dataset             
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# results = trainer.evaluate()
# print("Evaluation Results:", results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
import torch
import numpy as np
import pandas as pd
import time
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm

# Assuming train_dataset, model, and device are already defined

batch_size = 16
dataloader = DataLoader(
    train_dataset,
    sampler=SequentialSampler(train_dataset),
    batch_size=batch_size
)

# Set model to evaluation mode
model.eval()

# Initialize lists to store embeddings and patient IDs
all_embeddings = []
all_patient_ids = []

# Track progress and timing
start_time = time.time()
total_batches = len(dataloader)
processed_batches = 0

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing batches"):
        # Move data to the specified device (CPU/GPU)
        input_ids = batch['input_ids'].to(device)
        attention_masks = batch['attention_mask'].to(device)
        patient_ids_batch = batch['patient_id'].numpy()

        # Forward pass to get hidden states
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            output_hidden_states=True
        )

        # Extract [CLS] token embeddings from the last hidden layer
        cls_embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy()

        # Append embeddings and patient IDs to lists
        all_embeddings.append(cls_embeddings)
        all_patient_ids.extend(patient_ids_batch)

        # Update processed batch count
        processed_batches += 1

        # Check elapsed time and print completion percentage every 30 seconds
        elapsed_time = time.time() - start_time
        if elapsed_time >= 60:
            completion_percentage = (processed_batches / total_batches) * 100
            print(f"Completion Percentage: {completion_percentage:.2f}%")
            start_time = time.time()  # Reset timer

# Concatenate all embeddings into a single NumPy array
all_embeddings = np.concatenate(all_embeddings, axis=0)
all_patient_ids = np.array(all_patient_ids)

# Create a DataFrame to store embeddings and patient IDs
embeddings_df = pd.DataFrame(all_embeddings)
embeddings_df['patient_id'] = all_patient_ids

# Aggregate embeddings by patient ID (e.g., calculate mean embeddings)
patient_embeddings = embeddings_df.groupby('patient_id').mean().reset_index()

# Print completion message
print("Processing completed.")

# patient_embeddings now contains the aggregated embeddings for each patient


Processing batches:   1%|▎                                                        | 640/97770 [01:00<2:30:07, 10.78it/s]

Completion Percentage: 0.65%


Processing batches:   1%|▋                                                       | 1288/97770 [02:00<2:29:00, 10.79it/s]

Completion Percentage: 1.32%


Processing batches:   2%|█                                                       | 1936/97770 [03:00<2:28:07, 10.78it/s]

...

Completion Percentage: 99.32%


Processing batches: 100%|██████████████████████████████████████████████████████▉| 97754/97770 [2:31:07<00:01, 10.76it/s]

Completion Percentage: 99.98%


Processing batches: 100%|███████████████████████████████████████████████████████| 97770/97770 [2:31:08<00:00, 10.78it/s]


Processing completed.


In [18]:
patient_embeddings.shape

(45105, 769)

In [19]:
patient_embeddings.to_csv('patient_embeddings.csv', index=False)

## Handling the imbalance of Mortality

In [ ]:

from sklearn.utils import resample
train_data = pd.DataFrame({'text': train_texts, 'label': train_labels})
df_majority = train_data[train_data.label == 0]
df_minority = train_data[train_data.label == 1]
df_minority_oversampled = resample(
    df_minority,
    replace=True,
    n_samples=len(df_majority),
    random_state=42
)


In [ ]:
train_data_balanced = pd.concat([df_majority, df_minority_oversampled])
train_data_balanced = train_data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
train_texts_balanced = train_data_balanced['text']
train_labels_balanced = train_data_balanced['label']

train_encodings_balanced = tokenizer(
    train_texts_balanced.tolist(),
    truncation=True,
    padding=True

)
train_dataset_balanced = MimicDataset(train_encodings_balanced, train_labels_balanced)



In [ ]:

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels.values  # Use original labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define custom loss function
import torch.nn as nn

def custom_loss_fn(logits, labels):
    loss_fct = nn.CrossEntropyLoss(weight=class_weights)
    return loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))

# Subclass Trainer
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Initialize the model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)
model.to(device)

# Initialize the Trainer with your custom class
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_balanced,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()
